# Micro tutorial on how to run and scale hyperparameter optimization with LightGBM and Tune
<img src="https://docs.ray.io/en/latest/_images/tune_overview.png" alt="Tune and integrations" width="500">

Aug 2022. San Francisco, CA

## Preliminaries

### Imports

In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.datasets import load_wine
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

### Prepare dataset

In [ ]:
X, y = load_wine(return_X_y=True)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=7707, stratify=y
)

train_data = lgb.Dataset(data=X_train, label=y_train)
valid_data = lgb.Dataset(data=X_valid, label=y_valid, reference=train_data)

Here, we use [wine dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) (classification) and create LightGBM Dataset object that will be used for training and validation.

## Part 1: single LightGBM training session
<img src="https://lightgbm.readthedocs.io/en/latest/_images/LightGBM_logo_black_text.svg" alt="LightGBM Logo" width="500">

### Set training parameters for single training run

In [ ]:
training_parameters = {
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class": 3,
    "num_leaves": 7,
    "learning_rate": 0.1,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "max_depth": 5,
    "verbose": -1,
}

### Train LightGBM model and log results to stdout

In [ ]:
gbm = lgb.train(
    params=training_parameters,
    train_set=train_data,
    num_boost_round=10,
    valid_sets=[train_data, valid_data],
    valid_names=["train", "valid"],
    callbacks=[lgb.log_evaluation(period=10)],
)

### Report accuracy on validation data

In [ ]:
y_pred = np.argmax(gbm.predict(X_valid), axis=1)
acc = accuracy_score(y_true=y_valid, y_pred=y_pred)
print(f"Accuracy on valid set: {acc:.4f}")

### Summary
* We just ran single LightGBM training session. To do that we prepared dataset and training hyperparameters.
* Next, let's have a closer look at Tune.

## Part 2: Tune quickstart
<img src="https://docs.ray.io/en/latest/_images/tune.png" alt="Tune logo" width="500">

### Introduction to Tune
There are few components that we should look at first:

<img src="https://docs.ray.io/en/latest/_images/tune_flow.png" alt="Tune key concepts" width="800">

Learn more about it from the [Key concepts](https://docs.ray.io/en/latest/tune/key-concepts.html) docs page.

### Initialize Ray cluster

This cluster will be used for all tuning jobs.

In [ ]:
import ray

if ray.is_initialized:
    ray.shutdown()
cluster_info = ray.init()
cluster_info.address_info

### Import Tune

In [ ]:
from ray import tune

### Define search space

In [ ]:
search_space = {
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class": 3,
    "num_leaves": tune.choice([2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40, 100]),
    "learning_rate": tune.loguniform(1e-4, 1e-1),
    "feature_fraction": tune.uniform(0.85, 0.999),
    "bagging_fraction": 0.8,
    "bagging_freq": tune.randint(1, 11),
    "max_depth": tune.randint(1, 11),
    "verbose": -1,
}

### Define trainable

In [ ]:
def train_lgbm(training_params, checkpoint_dir=None):
    train_data = lgb.Dataset(data=X_train, label=y_train)
    valid_data = lgb.Dataset(data=X_valid, label=y_valid, reference=train_data)

    # Train LightGBM model and log results to stdout
    gbm = lgb.train(
        params=training_params,
        train_set=train_data,
        num_boost_round=10,
        valid_sets=[train_data, valid_data],
        valid_names=["train", "valid"],
        callbacks=[lgb.log_evaluation(period=10)],
    )

    y_pred = np.argmax(gbm.predict(X_valid), axis=1)
    acc = accuracy_score(y_true=y_valid, y_pred=y_pred)

    # Send accuracy back to Tune
    tune.report(valid_acc=acc)

### Run hyperparameter tuning, single trial

In [ ]:
analysis = tune.run(train_lgbm, config=search_space)

### Display accuracy from the best trial

In [ ]:
df = analysis.dataframe(metric="valid_acc")
df

### Summary
* We just ran first trial using Tune.
* Next, we will modify `tune.run()` in order to run tuning with 300 trials.

## Part 3: Execute 300 tuning runs with Tune

### Run hyperparameter tuning

In [ ]:
analysis = tune.run(
    train_lgbm,
    config=search_space,
    num_samples=300,
    metric="valid_acc",
    resources_per_trial={"cpu": 1},
    verbose=1,
)

### Display accuracy from the best trials

In [ ]:
df = analysis.dataframe(metric="valid_acc")
df.sort_values(by=["valid_acc"], ascending=False).head(n=5)

## Part 4: Population Based Training with Tune

### Introduction to Population Based Training
<img src="https://assets-global.website-files.com/621e749a546b7592125f38ed/62267281d1276db85e98c705_PBT%203.jpg" alt="Tune key concepts" width="500">

Learn more about Population based training of neural networks from the [blogpost](https://www.deepmind.com/blog/population-based-training-of-neural-networks) or [paper](https://arxiv.org/abs/1711.09846).

### Import Population Based Training from Tune schedulers

In [ ]:
from ray.tune.schedulers import PopulationBasedTraining

### Create Population Based Training scheduler

In [ ]:
pbt_scheduler = PopulationBasedTraining(
    time_attr="time_total_s",
    mode="max",
    perturbation_interval=3,
    hyperparam_mutations={
        "num_leaves": tune.choice([2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40, 100]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "feature_fraction": tune.uniform(0.85, 0.999),
        "bagging_freq": tune.randint(1, 11),
        "max_depth": tune.randint(1, 11),
    },
)

### Run hyperparameter tuning with Population Based Training

In [ ]:
analysis = tune.run(
    train_lgbm,
    config=search_space,
    num_samples=300,
    metric="valid_acc",
    resources_per_trial={"cpu": 1},
    scheduler=pbt_scheduler,
    verbose=1,
)

### Display accuracy from the best trials

In [ ]:
df = analysis.dataframe(metric="valid_acc")
df.sort_values(by=["valid_acc"], ascending=False).head(n=5)

### Summary
* We ran hyperparameter tuning with 300 trials using Population based Training. 

## Shutdown Ray cluster
Shutdown ray cluster at the end of the tutorial.

In [ ]:
ray.shutdown()

## Where to go next?

Congrats!

You just finished the micro tutorial on how to run and scale hyperparameter optimization with LightGBM and Tune.

Now, please go to the [micro tutorial README](https://github.com/kamil-kaczmarek/ray-tune-micro-tutorial/blob/kk/dev/README.md), to learn more about next steps, and options to reach out and connect with the community.